# Tutorial 7 : Neural Process Graphs

Last Update : 28 July 2019

**Aim**: 


In [1]:
N_THREADS = 12
# Nota Bene : notebooks don't deallocate GPU memory
IS_FORCE_CPU = True # can also be set in the trainer

## Environment

In [2]:
cd ..

/conv


In [3]:
%autosave 600
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# CENTER PLOTS
from IPython.core.display import HTML
display(HTML(""" <style> .output_png {display: table-cell; text-align: center; margin:auto; }
.prompt display:none;}  </style>"""))

import os
if IS_FORCE_CPU:
    os.environ['CUDA_VISIBLE_DEVICES'] = ""
    
import sys
sys.path.append("notebooks")

import numpy as np
import matplotlib.pyplot as plt
import torch
torch.set_num_threads(N_THREADS)

Autosaving every 600 seconds


# Dataset 

Cora, Citeseer, Pubmed
PROTEINS, enzymes


In [4]:
from torch_geometric.datasets import TUDataset, Planetoid, PPI, QM7b, ModelNet
from sklearn.preprocessing import StandardScaler

In [5]:
from random import shuffle

datasets=dict(enzymes = TUDataset(root='data/ENZYMES', name='ENZYMES', use_node_attr=True, ),
             proteins = TUDataset(root='data/PROTEINS_full', name='PROTEINS_full', use_node_attr=True),
             synthie = TUDataset(root='data/Synthie', name='Synthie', use_node_attr=True))

data_specific_kwargs = {k:dict(y_dim=d.num_node_features,
                               t_dim=d.num_classes
                              ) 
                        for k,d in datasets.items()}

def train_test_split(d, transform=lambda x : StandardScaler().fit_transform(x)):
    if transform is not None:
        d.data.x = torch.from_numpy(transform(d.data.x.numpy()))
    d = d.shuffle()
    return list(d[:-len(d) // 10]), list(d[-len(d) // 10:])

# should store the split
#datasets = {k:train_test_split(d) for k,d in datasets.items()}

In [6]:
from skssl.transformers.neuralproc.datasplit import GridCntxtTrgtGetter, RandomMasker, no_masker, half_masker
from utils.data.tsdata import get_timeseries_dataset, SparseMultiTimeSeriesDataset

get_cntxt_trgt_test = GridCntxtTrgtGetter(context_masker=RandomMasker(min_nnz=0.01, max_nnz=0.50),
                                         target_masker=no_masker,
                                         is_add_cntxts_to_trgts=False, is_stratify=True)  # don't context points to tagrtes

get_cntxt_trgt_feat = GridCntxtTrgtGetter(context_masker=no_masker,
                                     target_masker=no_masker,
                                     is_add_cntxts_to_trgts=False)  # don't context points to tagrtes

get_cntxt_trgt = GridCntxtTrgtGetter(context_masker=RandomMasker(min_nnz=0.01, max_nnz=0.99),
                                 target_masker=RandomMasker(min_nnz=0.50, max_nnz=0.99),
                                 is_add_cntxts_to_trgts=False, is_stratify=True)  # don't context points to tagrtes

import torch
import skorch
from torch_geometric.data import Batch

def cntxt_trgt_collate(get_cntxt_trgt, is_repeat_batch=False):
    def mycollate(data_list):
        
        if is_repeat_batch:
            data_list = data_list + data_list
            
        data = Batch.from_data_list(data_list, [])
        edge_attr = torch.ones_like(data.edge_index[0], dtype=torch.float) if data.edge_attr is None else data.edge_attr
        
        X, mask_context, mask_target = get_cntxt_trgt(data.x.t().unsqueeze(0), None, is_grided=True, 
                                                      stratify=data.batch)
        data.x = X.squeeze(0).t()
            
        
        # Can't pass a Dataset directly, since it expects tensors. 
        # Use dict of tensors instead. Also, use torch.sparse for 
        # adjacency matrix to pass skorch's same-dimension check
        return {
            "X":{'x': data.x,
            'adj': torch.sparse.FloatTensor(data.edge_index, 
                                            edge_attr, 
                                            size=[data.num_nodes, data.num_nodes], 
                                            device=data.x.device),
            'batch': data.batch},
            'mask_context':mask_context.squeeze(0),
            'mask_target':mask_target.squeeze(0),
            
        }, data.y
    
    return mycollate
        
class SkorchDataset(skorch.dataset.Dataset):
    def __init__(self, X, y):
        # We need to specify `length` to avoid checks
        super(SkorchDataset, self).__init__(X, y, length=len(X))
    
    def transform(self, X, y):
        return X   # Ignore y, since it is included in X
    

In [7]:
X_DIM = 2  # 2D spatial input 
#Y_DIM = data.shape[0]
N_TARGETS = None#data.n_classes

#label_percentages = [N_TARGETS, N_TARGETS*2, 0.01, 0.05, 0.1, 0.3, 0.5, 1]

# Model

In [8]:
from skssl.transformers import GraphConvNeuralProcess, GraphNeuralProcessLoss
from skssl.predefined import GCN, UnetGCN, GAT
from skssl.transformers.neuralproc.datasplit import precomputed_cntxt_trgt_split
from functools import partial
from torch_geometric.nn import GCNConv, GINConv, global_mean_pool
from torch_geometric.nn import GraphConv, TopKPooling
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn as nn
import torch.nn.functional as F
from types import SimpleNamespace

models = {}

class Topk(torch.nn.Module):
    def __init__(self, y_dim, t_dim):
        super().__init__()

        self.conv1 = GraphConv(y_dim, 128)
        self.pool1 = TopKPooling(128, ratio=0.8)
        self.conv2 = GraphConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.8)
        self.conv3 = GraphConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.8)

        self.lin1 = torch.nn.Linear(256, 128)
        self.lin2 = torch.nn.Linear(128, 64)
        self.lin3 = torch.nn.Linear(64, t_dim)

    def forward(self, x, adj, batch):
        edge_index = adj._indices()

        x = F.relu(self.conv1(x, edge_index))
        x, edge_index, _, batch, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
        x, edge_index, _, batch, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))
        x, edge_index, _, batch, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)

        return F.relu(self.lin2(x))


class GATSupervised(torch.nn.Module):
    def __init__(self, y_dim, t_dim, dim=8, heads=8, dropout=0.6, n_layers=2):
        super().__init__()
        self.gat = GAT(y_dim, out_channels=32, dim=dim, heads=heads, dropout=dropout, n_layers=n_layers)
        self.lin = nn.Linear(32, t_dim)
        
    def forward(self, x, adj, batch):
        x = self.gat(SimpleNamespace(x=x, edge_index=adj._indices(), batch=batch))
        x = global_mean_pool(x.x, x.batch)
        return self.lin(x) 
    
class GIN0(torch.nn.Module):
    def __init__(self, y_dim, t_dim, num_layers=5, hidden=32):
        super(GIN0, self).__init__()
        self.conv1 = GINConv(
            nn.Sequential(
            nn.Linear(y_dim, hidden),
                nn.ReLU(),
                nn.Linear(hidden, hidden),
                nn.ReLU(),
                nn.BatchNorm1d(hidden),
            ),
            train_eps=False)
        self.convs = torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(
                GINConv(
                    nn.Sequential(
                        nn.Linear(hidden, hidden),
                        nn.ReLU(),
                        nn.Linear(hidden, hidden),
                        nn.ReLU(),
                        nn.BatchNorm1d(hidden),
                    ),
                    train_eps=False))
        self.lin1 = torch.nn.Linear(num_layers * hidden, hidden)
        self.lin2 = nn.Linear(hidden, t_dim)

    def reset_parameters(self):
        self.conv1.reset_parameters()
        for conv in self.convs:
            conv.reset_parameters()
        self.lin1.reset_parameters()
        self.lin2.reset_parameters()

    def forward(self, x, adj, batch):
        edge_index = adj._indices()
        x = self.conv1(x, edge_index)
        xs = [x]
        for conv in self.convs:
            x = conv(x, edge_index)
            xs += [x]
        x = global_mean_pool(torch.cat(xs, dim=1), batch)
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        return self.lin2(x)

    def __repr__(self):
        return self.__class__.__name__

# initialize one model for each dataset
models["supervised_gat_large"] = lambda y_dim, t_dim : GATSupervised(y_dim, t_dim, dim=16, n_layers=3)
models["supervised_gin0_large"] = lambda y_dim, t_dim : GIN0(y_dim, t_dim, hidden=64)
models["supervised_topk"] = lambda y_dim, t_dim : Topk(y_dim, t_dim)
#models["supervised_gat_small"] = lambda y_dim, t_dim : GATSupervised(y_dim, t_dim, dim=8, n_layers=2)
#models["supervised_gin0_small"] = lambda y_dim, t_dim : GIN0(y_dim, t_dim, hidden=32)

In [9]:
from utils.helpers import count_parameters
for k,v in models.items():
    print(k, "- N Param:", count_parameters(v(y_dim=20, t_dim=5)))

supervised_gat_large - N Param: 53189
supervised_gin0_large - N Param: 60293
supervised_topk - N Param: 112901


In [10]:
from skssl.transformers.neuralproc.datasplit import GridCntxtTrgtGetter, RandomMasker, no_masker, half_masker
from utils.data.tsdata import get_timeseries_dataset, SparseMultiTimeSeriesDataset
import torch
import skorch
from torch_geometric.data import Batch
class SkorchDataLoader(torch.utils.data.DataLoader):
    def _collate_fn(self, data_list, follow_batch=[]):
        data = Batch.from_data_list(data_list, follow_batch)
        edge_attr = torch.ones_like(data.edge_index[0], dtype=torch.float) if data.edge_attr is None else data.edge_attr
        
        # Can't pass a Dataset directly, since it expects tensors. 
        # Use dict of tensors instead. Also, use torch.sparse for 
        # adjacency matrix to pass skorch's same-dimension check
        return {
            'x': data.x,
            'adj': torch.sparse.FloatTensor(data.edge_index, 
                                            edge_attr, 
                                            size=[data.num_nodes, data.num_nodes], 
                                            device=data.x.device),
            'batch': data.batch
        }, data.y
    
    def __init__(self,
                 dataset,
                 batch_size=1,
                 shuffle=True,
                 follow_batch=[],
                 **kwargs):
        super(SkorchDataLoader, self).__init__(
            dataset,
            batch_size,
            shuffle,
            collate_fn=lambda data_list: self._collate_fn(data_list, follow_batch),
            **kwargs)
        
class SkorchDataset(skorch.dataset.Dataset):
    def __init__(self, X, y):
        # We need to specify `length` to avoid checks
        super(SkorchDataset, self).__init__(X, y, length=len(X))
    
    def transform(self, X, y):
        return X   # Ignore y, since it is included in X

# Training

In [11]:
N_EPOCHS = 100 
BATCH_SIZE = 32
IS_RETRAIN = True # if false load precomputed
chckpnt_dirname="results/notebooks/neural_process_graph/"

from ntbks_helpers import train_models_

In [ ]:
data_trainers = {}
data_keys = datasets.keys()


for run in range(10):
    for name_mod in models.keys():
        for data_name in data_keys:

            datasets=dict(enzymes = TUDataset(root='data/ENZYMES', name='ENZYMES', use_node_attr=True),
                         proteins = TUDataset(root='data/PROTEINS_full', name='PROTEINS_full', use_node_attr=True),
                     synthie = TUDataset(root='data/Synthie', name='Synthie', use_node_attr=True))

            data_specific_kwargs = {k:dict(y_dim=d.num_node_features,
                                           t_dim=d.num_classes
                                          ) 
                                    for k,d in datasets.items()}
            # should store the split
            datasets = {k:train_test_split(d) for k,d in datasets.items() if k == data_name}


            data_trainers.update(train_models_({k:d for k,d in datasets.items() if "enzymes" in k}, 
                                   {k+ "_run{}".format(run):m  for k,m in models.items() if name_mod == k},
                                  data_specific_kwargs=data_specific_kwargs,
                                 patience=15,
                                 chckpnt_dirname=chckpnt_dirname,
                                  max_epochs=N_EPOCHS,
                                  batch_size=BATCH_SIZE,
                                  is_retrain=IS_RETRAIN,
                                  callbacks=[],
                                  iterator_train=SkorchDataLoader,
                                iterator_valid=SkorchDataLoader,
                                dataset=SkorchDataset,
                                               is_monitor_acc=True,
                                  mode="classifier"))


--- Training enzymes/supervised_gat_large_run0 ---



  epoch    train_loss    valid_acc    valid_loss    cp     dur
-------  ------------  -----------  ------------  ----  ------
      1        1.7818       0.1167        1.8072     +  4.3235


      2        1.7397       0.2167        1.7869     +  5.2558


      3        1.6972       0.1833        1.7583        5.1891


      4        1.6620       0.1500        1.8081        4.7881


...

     51        0.1485       0.6833        1.9838        2.2297


     52        0.1068       0.6500        2.1180        2.2168


     53        0.1152       0.6500        2.3206        2.2016


     54        0.1592       0.6833        2.0253        2.3125


     55        0.1104       0.6833        2.1021        2.3168


In [ ]:
for k,t in data_trainers.items(): 
    for e, h in enumerate(t.history[::-1]):
        if h["valid_loss_best"]:
            print(k, "epoch:", len(t.history)-e, 
                  "val_loss:", h["valid_loss"], 
                 "valid_acc", h["valid_acc"])
            break

In [ ]:
import pandas as pd

In [ ]:
import pandas as pd
data_trainers["enzymes/supervised_gat_large_run0"].history[-1]["valid_acc"]

In [ ]:
import pandas as pd
data_trainers["enzymes/supervised_gat_large_run0"].history[-1]["valid_acc"]

out = pd.Series({k:v.history[-1]["valid_acc"] for k,v in data_trainers.items()}).reset_index(name="accuracy")
splitted = out["index"].str.split("/", expand = True)
out["data"] = splitted[0]
out["models"] = splitted[1]

splitted2 = out["models"].str.split("_run", expand = True)
out["models"] = splitted2[0]
out["run"] = splitted2[1]


out.drop(columns =["index"], inplace = True) 

out.groupby(["data", "models"]).describe()

In [ ]:
out.groupby(["data", "models"]).describe()